## Import libraries

In [1]:
import os
import pandas as pd
import numpy as np
import copy
import torch
import loss
from torch import optim
from metrics import eval_metrics, get_epoch_acc
from dataloader import DataLoader
from cross_val import CrossVal
from torchvision import transforms
from eval import eval
from config import ModelParameters
from PIL import Image
import cv2
import tensorflow as tf

# Import available models, you can also explore other PyTorch models
from cracknet import cracknet
from unet import UNet, UNetResnet
from segnet import SegNet, SegResNet

/home/participant4/venv/lib/python3.10/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


ModuleNotFoundError: No module named 'tensorflow'

In [4]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
os.environ["TORCH_USE_CUDA_DSA"] = "1"

## Training functions

In [5]:
def train_oneepoch(model, class_count, criterion, eval_metric, device, my_optimizer, my_lr_scheduler, dataloader):
    model.train()
    batch_loss = 0
    batch_acc_numerator = 0
    batch_acc_denominator = 0
    epoch_lr = my_lr_scheduler.get_last_lr()[0]
    for inputs, labels in dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        my_optimizer.zero_grad()
        #print(inputs.size())
        mask_pred = model(inputs)
        loss = criterion(mask_pred, labels)
        loss.backward()
        my_optimizer.step()
        
        # batch_loss += loss
        batch_loss += loss.item()
        
        batch_acc_numerator_tmp, batch_acc_denominator_tmp = eval_metrics(mask_pred, labels, class_count, eval_metric)
        batch_acc_numerator += batch_acc_numerator_tmp
        batch_acc_denominator += batch_acc_denominator_tmp
    my_lr_scheduler.step()
    epoch_loss = batch_loss / len(dataloader)
    epoch_acc = get_epoch_acc(batch_acc_numerator, batch_acc_denominator, eval_metric)

    return epoch_loss, epoch_acc, epoch_lr

In [6]:
def train_main(model, class_count, criterion, eval_metric, EPOCHS, DEVICE, my_optimizer, my_lr_scheduler=None, dataloaders=None, logging=False, model_name='model.pt'):
    model.to(DEVICE)
    train_loss = []
    train_acc = []
    val_loss = []
    val_acc = []
    lr = []
    best_val_acc = 0

    for epoch in range(EPOCHS):
        epoch_train_loss, epoch_train_acc, epoch_lr = train_oneepoch(model, class_count, criterion, eval_metric, DEVICE, my_optimizer, my_lr_scheduler, dataloaders[epoch%3]['train'])
        epoch_val_loss, epoch_val_acc = eval(model, class_count, criterion, eval_metric, DEVICE, dataloaders[epoch%3]['val'])

        if epoch_val_acc > best_val_acc:
            best_val_acc = epoch_val_acc
        best_state_dict = copy.deepcopy(model.state_dict())

        if logging:
            # train_loss.append(epoch_train_loss.detach().cpu().numpy().tolist())
            train_loss.append(epoch_train_loss)
            train_acc.append(epoch_train_acc)
            # val_loss.append(epoch_val_loss.detach().cpu().numpy().tolist())
            val_loss.append(epoch_val_loss)
            val_acc.append(epoch_val_acc)
            lr.append(epoch_lr)
        torch.cuda.empty_cache() 
        
        print(f'Epoch {epoch}/{EPOCHS - 1}: TrainLoss: {epoch_train_loss:.4f}, TrainAcc: {epoch_train_acc:.4f}, ValLoss: {epoch_val_loss:.4f}, ValAcc: {epoch_val_acc:.4f}')

    print('Best Acc: {:4f}'.format(best_val_acc))

    # load best model weights
    model.load_state_dict(best_state_dict)
    torch.save(model, model_name + '.pt')
    
    # save training details
    pd.DataFrame({'Epochs':range(EPOCHS), 'Learning Rate': lr, 'Training Loss': train_loss, 
                    'Training Acc': train_acc, 'Validation Loss': val_loss, 
                    'Validation Acc': val_acc}).to_csv(model_name + '.csv', index = False)

    return model

## Training parameters

Inside the data directory, the structure should be following:
- train
    - images
        - IL 991.png
        - IL 992.png
    - labels
        - IL 991.npy
        - IL 992.npy
    - class_names.txt
- val
    - images
        - IL 993.png
    - labels
        - IL 993.npy
    - class_names.txt

class_names.txt is to specify the label class name for the training

Example content of class_names.txt:

\_background_ <br>
fault

In [7]:
# Name the data directory and model filename
DIR = 'data/' # Data directory
MODEL_FILENAME = 'cracknet.pt' # Model filename



In [8]:
num_fault = []
num_horizon = []
num_seismic = []

for name in os.listdir('../data/aug_fault_mask'):
    if name == '.ipynb_checkpoints':
        continue
    code = name.replace("fault","").replace(".npy","")
    if os.path.isfile('../data/aug_horizon_mask/horizon{}.npy'.format(code)) and os.path.isfile('../data/aug_raw_seismic/seismic{}.png'.format(code)):
        num_fault.append(name)
        num_horizon.append('horizon{}.npy'.format(code))
        num_seismic.append('seismic{}.png'.format(code))
    

df = pd.DataFrame({
    'RAW_SEISMIC': [f"{x}" for x in num_seismic],
    'RAW_FAULT': [f"{x}" for x in num_fault],
    'RAW_HORIZON': [f"{x}" for x in num_horizon]
})

In [9]:
cv = CrossVal(df, 3)
dataloaders = cv
# dataset = {}
# dataset['train'] = LabelMe(data_folder=os.path.join(DIR,'train'), transform=data_transforms['train'],
#                                 img_size=(1024, 1024))
# dataset['val'] = LabelMe(data_folder=os.path.join(DIR,'val'), transform=data_transforms['val'],
#                                 img_size=(1024, 1024))
# dataloaders = {x: torch.utils.data.DataLoader(dataset[x], batch_size = BATCH_SIZE,
#                                             shuffle = True, num_workers = 8, 
#                                             drop_last = False)
#                                             for x in ['train', 'val']}
class_count = len(dataloaders[0]['train'].dataset.label)

In [10]:
# Choose a model for training, you can refer to the models that have been imported above
model = cracknet(pretrained = ModelParameters.PRETRAINED, num_classes = class_count)

my_optimizer = optim.Adam(model.parameters(), lr = ModelParameters.LEARNING_RATE) # Check https://pytorch.org/docs/stable/optim.html for other optimizers
my_lr_scheduler = optim.lr_scheduler.StepLR(my_optimizer, step_size=25, gamma=0.1) # Check https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate for other schedulers

## Start model training

In [ ]:
train_main(model, class_count, ModelParameters.CRITERION, ModelParameters.EVAL_METRIC, ModelParameters.EPOCHS, DEVICE, my_optimizer, my_lr_scheduler, dataloaders, logging = ModelParameters.LOGGING, model_name = MODEL_FILENAME)

########
torch.Size([6, 3, 512, 512])
########
########
torch.Size([6, 3, 512, 512])
########
########
torch.Size([6, 3, 512, 512])
########
########
torch.Size([6, 3, 512, 512])
########
########
torch.Size([6, 3, 512, 512])
########
########
torch.Size([6, 3, 512, 512])
########
########
torch.Size([6, 3, 512, 512])
########
########
torch.Size([6, 3, 512, 512])
########
########
torch.Size([6, 3, 512, 512])
########
########
torch.Size([6, 3, 512, 512])
########
########
torch.Size([6, 3, 512, 512])
########
########
torch.Size([6, 3, 512, 512])
########
########
torch.Size([6, 3, 512, 512])
########
########
torch.Size([6, 3, 512, 512])
########
########
torch.Size([6, 3, 512, 512])
########
########
torch.Size([6, 3, 512, 512])
########
########
torch.Size([6, 3, 512, 512])
########
########
torch.Size([6, 3, 512, 512])
########
########
torch.Size([6, 3, 512, 512])
########
########
torch.Size([6, 3, 512, 512])
########
########
torch.Size([6, 3, 512, 512])
########
########
torc

In [12]:
idx = 100
dl = DataLoader([],[],[], (512, 512))
img = []
img.append(dl.input_transforms(Image.open(os.path.join(dl.RAW_SEISMIC_FOLDER, df.at[idx, "RAW_SEISMIC"]))))
img.append(dl.input_transforms(Image.open(os.path.join(dl.RAW_SEISMIC_FOLDER, df.at[idx+1, "RAW_SEISMIC"]))))
img = torch.stack(img, dim=0)
# fault_name = '.'.join(self.img_name[idx].split('.', maxsplit = 1)[:-1]) + '.npy'
# fault = np.load(os.path.join(self.RAW_FAULT_FOLDER, self.fault_files[idx]), allow_pickle=True).astype(np.uint8)
# fault = cv2.resize(mask, self.img_size, cv2.INTER_AREA)
# fault_img = Image.open(os.path.join(self.RAW_FAULT_FOLDER, self.fault_files[idx]))
# fault = self.output_transforms(fault_img)
# fault  = np.array(fault_img)
# mask_name = '.'.join(self.img_name[idx].split('.', maxsplit = 1)[:-1]) + '.npy'
mask = []
mask.append(torch.from_numpy(cv2.resize(np.load(os.path.join(dl.FAULT_MASK_FOLDER, df.at[idx, "RAW_FAULT"])).astype(np.uint8), dl.IMAGE_SIZE, cv2.INTER_AREA).astype(np.int64)))
mask.append(torch.from_numpy(cv2.resize(np.load(os.path.join(dl.FAULT_MASK_FOLDER, df.at[idx+1, "RAW_FAULT"])).astype(np.uint8), dl.IMAGE_SIZE, cv2.INTER_AREA).astype(np.int64)))
mask = torch.stack(mask, dim=0)
img = img.to(DEVICE)
mask = mask.to(DEVICE)
mask_pred = model(img)

In [13]:
ModelParameters.CRITERION(mask_pred, mask)

tensor(0.2678, device='cuda:0', grad_fn=<RsubBackward1>)

In [14]:
eval_metrics(mask_pred, mask, class_count,'batch_pix_accuracy')

(array(524288), array(524288))

In [113]:
_, predict = torch.max(mask_pred.data, 1)
# predict = predict + 1
# mask = mask + 1
# labeled = (mask > 0) * (mask <= class_count)
# pixel_labeled = labeled.sum()
# pixel_correct = ((predict == mask) * labeled).sum()
# (pixel_correct.cpu().numpy(), pixel_labeled.cpu().numpy())

In [129]:
mask_pred.data[:,0]

tensor([[[3.7269, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 3.7026],
         [3.7230, 3.7357, 3.7314,  ..., 3.7001, 3.7036, 0.0000],
         [3.7155, 3.7183, 3.7125,  ..., 0.0000, 0.0000, 3.7095],
         ...,
         [3.7073, 3.7086, 3.7009,  ..., 0.0000, 3.7119, 0.0000],
         [0.0000, 3.7048, 0.0000,  ..., 3.7084, 3.7062, 3.6991],
         [0.0000, 3.6966, 3.7024,  ..., 3.7000, 0.0000, 0.0000]],

        [[3.7173, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 3.7102],
         [3.7153, 3.7252, 3.7193,  ..., 3.7127, 3.7187, 0.0000],
         [3.7151, 3.7186, 3.7075,  ..., 0.0000, 0.0000, 3.7166],
         ...,
         [3.7114, 3.7085, 3.7084,  ..., 0.0000, 3.7113, 0.0000],
         [0.0000, 3.7042, 0.0000,  ..., 3.7132, 3.7225, 3.6966],
         [0.0000, 3.7061, 3.7064,  ..., 3.7055, 0.0000, 0.0000]]],
       device='cuda:0')

In [120]:
predict.sum()

tensor(0, device='cuda:0')

In [ ]:
cracknet_model = cracknet(pretrained = ModelParameters.PRETRAINED, num_classes = class_count)
cracknet_model_total_params = sum(p.numel() for p in cracknet_model.parameters())
cracknet_model_total_params

unet_model = UNet(num_classes = class_count)
unet_model_total_params = sum(p.numel() for p in unet_model.parameters())
unet_resnet_model = UNetResnet(num_classes = class_count)
unet_resnet_model_total_params = sum(p.numel() for p in unet_resnet_model.parameters())

segnet_model = SegNet(num_classes = class_count)
segnet_model_total_params  = sum(p.numel() for p in segnet_model.parameters())
segresnet_model = SegResNet(num_classes = class_count)
segresnet_model_total_params  = sum(p.numel() for p in segresnet_model.parameters())

In [11]:
cracknet_model_total_params

49320390

In [12]:
unet_model_total_params

26355234

In [13]:
unet_resnet_model_total_params

29999920

In [14]:
segnet_model_total_params

16310850

In [15]:
segresnet_model_total_params

53553346

In [16]:
cracknet_model_total_params/segnet_model_total_params

3.023778037318717